<a href="https://colab.research.google.com/github/BigTMiami/AdaptOrDie/blob/main/adapters_classification_no_pretraining_20_percent_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Summary
This tests using an adapter for pretraining

# Things to change in real run
* Use the proper datasets (not dev for validation?)




# Setup

In [1]:
# from google.colab import drive
# drive.mount("/content/drive")
# %cd '/content/drive/MyDrive/AdaptOrDie'

In [2]:
!pip install datasets
!pip install transformers[torch]
!pip install adapters
!pip install scikit-learn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 14.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.4/297.4 kB 5.2 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x8

In [3]:
!pip install pynvml

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 kB 2.2 MB/s eta 0:00:00


In [4]:
from pynvml import *

def print_gpu_utilization():
    nvmlInit()
    handle = nvmlDeviceGetHandleByIndex(0)
    info = nvmlDeviceGetMemoryInfo(handle)
    gpu_used = info.used//1024**2
    print(f"GPU {gpu_used} MB")

print_gpu_utilization()

GPU 448 MB


In [5]:
from sklearn.metrics import accuracy_score, f1_score

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)

    # Calculate accuracy
    accuracy = accuracy_score(labels, preds)

   # Calculate precision, recall, and F1-score
    f1 = f1_score(labels, preds, average='macro')

    return {
        'accuracy': accuracy,
        'f1_macro': f1
    }

In [6]:
from datasets import load_dataset
dataset_name = "BigTMiami/amazon_helpfulness"
train_dataset = load_dataset(dataset_name, split="train[:20%]")
eval_dataset = load_dataset(dataset_name, split="dev")

print(train_dataset)
print(eval_dataset)

print_gpu_utilization()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/repocard.py:105: UserWarning: Repo card metadata block was not found. Setting CardData to empty.
  warnings.warn("Repo card metadata block was not found. Setting CardData to empty.")


Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Generating dev split: 0 examples [00:00, ? examples/s]

Dataset({
    features: ['label', 'input_ids', 'attention_mask'],
    num_rows: 23050
})
Dataset({
    features: ['label', 'input_ids', 'attention_mask'],
    num_rows: 5000
})
GPU 448 MB


In [7]:
from transformers import RobertaConfig
from adapters import AutoAdapterModel

config = RobertaConfig.from_pretrained("roberta-base")
model = AutoAdapterModel.from_pretrained(
    "roberta-base",
    config=config,
)
print_gpu_utilization()

/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaAdapterModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.weight', 'heads.default.3.bias', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


GPU 448 MB


In [8]:
# adapter_hub_name = "adapter_classifier_from_prtrained_test"
# adapter_name = model.load_adapter("BigTMiami/domain_adapter_test")
# # Add head for masked language modeling
# model.add_classification_head(
#     adapter_name,
#     num_labels=2,
#     id2label={ 0: "unhelpful", 1: "helpful"}, overwrite_ok=True
#   )

# # Set the adapter to be used for training
# model.train_adapter(adapter_name)

In [10]:
adapter_hub_name = "adapter_classification_no_pretraining_20_percent"
adapter_name = "classifier_seq_bn_no_pretraining_20_percent"
adapter_type = "seq_bn" # could be "lora", etc.

# Add a new adapter
model.add_adapter(adapter_name, config=adapter_type)

# Add head for classification modeling
model.add_classification_head(
    adapter_name,
    num_labels=2,
    id2label={ 0: "unhelpful", 1: "helpful"})

# Set the adapter to be used for training
model.train_adapter(adapter_name)

In [11]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("roberta-base")
print_gpu_utilization()

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

GPU 448 MB


In [12]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

# Train

In [16]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir=adapter_hub_name,
    learning_rate=1e-4,
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    logging_steps=250,
    overwrite_output_dir=True,
    # The next line is important to ensure the dataset labels are properly passed to the model
    remove_unused_columns=False,
)

In [17]:
from adapters import AdapterTrainer

trainer = AdapterTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)
print_gpu_utilization()

GPU 12255 MB


/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False)
  warnings.warn(


In [18]:
training_results = trainer.train()
print_gpu_utilization()
print(training_results)

Step,Training Loss
250,0.357000
500,0.355500
750,0.370000
1000,0.334200
1250,0.324800
1500,0.354200
1750,0.328700
2000,0.334400
2250,0.327400
2500,0.325700


GPU 12259 MB
TrainOutput(global_step=4323, training_loss=0.32946147649563384, metrics={'train_runtime': 1109.2582, 'train_samples_per_second': 62.339, 'train_steps_per_second': 3.897, 'total_flos': 1.7389360436066208e+16, 'train_loss': 0.32946147649563384, 'epoch': 3.0})


In [19]:
eval_results = trainer.evaluate()
trained_eval_loss = eval_results["eval_loss"]
print_gpu_utilization()
print(f"Trained Eval Loss: {trained_eval_loss:.2f}")
print(eval_results)

GPU 12259 MB
Trained Eval Loss: 0.33
{'eval_loss': 0.33062857389450073, 'eval_accuracy': 0.8662, 'eval_f1_macro': 0.6273391985352926, 'eval_runtime': 37.7912, 'eval_samples_per_second': 132.306, 'eval_steps_per_second': 8.282, 'epoch': 3.0}


In [20]:
eval_results = trainer.evaluate()
eval_loss = eval_results["eval_loss"]
eval_f1_macro = eval_results["eval_f1_macro"]
print_gpu_utilization()
print(f"Trained eval_f1_macro:{100.0 * eval_f1_macro:.2f} Eval Loss: {eval_loss:.2f}")
print(eval_results)

GPU 12259 MB
Trained eval_f1_macro:62.73 Eval Loss: 0.33
{'eval_loss': 0.33062857389450073, 'eval_accuracy': 0.8662, 'eval_f1_macro': 0.6273391985352926, 'eval_runtime': 37.6446, 'eval_samples_per_second': 132.821, 'eval_steps_per_second': 8.315, 'epoch': 3.0}


In [ ]:
model.push_adapter_to_hub(
    adapter_hub_name,
    adapter_name,
    datasets_tag=dataset_name
)

pytorch_adapter.bin:   0%|          | 0.00/3.60M [00:00<?, ?B/s]

pytorch_model_head.bin:   0%|          | 0.00/2.37M [00:00<?, ?B/s]

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

CommitInfo(commit_url='https://huggingface.co/BigTMiami/adapter_classification_no_pretraining_20_percent/commit/8dbb4d8dbb41db6c52e444c3b56f69fab6e78f07', commit_message='Upload model', commit_description='', oid='8dbb4d8dbb41db6c52e444c3b56f69fab6e78f07', pr_url=None, pr_revision=None, pr_num=None)

In [ ]:
print("Disconnecting Session")
from google.colab import runtime
runtime.unassign()